## 在运行本笔记本之前，请先下载以下工具：

1. **查找当前的 Google Chrome 版本**  
   - 打开 Google Chrome，并在地址栏输入：  
     ```
     chrome://settings/help
     ```
   - 建议使用 **133** 版本的 Google Chrome（或最新可用的稳定版本）。

2. **下载对应版本的 ChromeDriver**  
   - 前往官方的 Chrome 测试下载页面：  
     [https://googlechromelabs.github.io/chrome-for-testing/#stable](https://googlechromelabs.github.io/chrome-for-testing/#stable)  
   - 下载与您的 Chrome 版本相匹配的 ChromeDriver。  
   - 例如，如果您的 Chrome 版本是 133，请下载 **ChromeDriver 133**。

3. **查找已下载的 ChromeDriver**  
   - 如果您使用 macOS，可在终端 (Terminal) 中运行以下命令来查找 `chromedriver` 的位置：
     ```bash
     mdfind -name chromedriver
     ```
   - 如果使用其他操作系统，请检查默认的 **下载** 目录或您保存该文件的目录。

In [6]:
# 导入必要的包
import os
import re
import time
import json
import random
import pymongo
import pandas as pd
import requests  # 新增：用于下载图片

from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm.notebook import tqdm

from scrapy.selector import Selector
from scrapy.http import TextResponse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

##### 启动 Chrome 浏览器实例：

打开 **terminal**, 下载 Chrome Driver (假的 Google Chrome)

```bash
brew install chromedriver
chmod +x /opt/homebrew/bin/chromedriver
```

输入以下命令（将 `your Chrome.exe path` 替换为您的 Google Chrome 浏览器路径）：
```bash
<your Chrome.exe path> --remote-debugging-port=9222 --user-data-dir="/Users/<your home folder name>/selenium/AutomationProfile"
```

- 请将your Chrome.exe path替换为您的Chrome浏览器所在路径，例如<br>`C:\Program Files\Google\Chrome\Application\chrome.exe`
- 配置 chromedriver 相关信息，请参考官方文档：[ChromeDriver](https://developer.chrome.com/docs/chromedriver)
- 来做个比方， 我的 *terminal command* 会是:

/Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome \
  --remote-debugging-port=9222 \
  --user-data-dir="/Users/princess/selenium/AutomationProfile"

In [7]:
# 配置Chrome浏览器
options = Options()
options.add_experimental_option('debuggerAddress', '127.0.0.1:9222')
options.add_argument('--incognito')
browser = webdriver.Chrome(options=options)
action = ActionChains(browser)

In [8]:
key_word = ""
num = 0

def check_login_status(browser):
    print("即将开始检查小红书登录状态...")
    print("爬取数据有账户封禁的风险，建议使用非主账号登录。")
    
    while True:
        page_source = browser.page_source
        if '登录探索更多内容' in page_source:
            print('暂未登录，请手动登录')
            print('检查时间:', time.ctime())
            time.sleep(10)
        else:
            print('登录成功')
            print('检查时间:', time.ctime())
            time.sleep(3)
            break

def check_page_load_status(browser, keyword):
    print("即将开始检查网页加载状态...")
    print("如果网页进入人机验证页面，请先手动完成验证。")
    
    while True:
        if keyword in browser.title:
            print('加载成功')
            print('检查时间:', time.ctime())
            break
        else:
            time.sleep(2)

def selenium_test():
    """
    登录状态检查，网页加载检查，根据用户输入进行搜索
    """
    global key_word, num
    browser.get('https://www.xiaohongshu.com/explore')
    
    check_login_status(browser)
    
    print("请在文本框中根据提示输入搜索关键词和笔记爬取数量。")
    keyword = input("搜索关键词：")
    key_word = keyword
    
    try:
        num = int(input("笔记爬取数量："))
    except ValueError:
        print("请输入有效的整数作为爬取数量。")
        return
    
    url = f'https://www.xiaohongshu.com/search_result?keyword={keyword}&source=web_explore_feed'
    browser.get(url)
    time.sleep(3)
    
    check_page_load_status(browser, keyword)

selenium_test()

即将开始检查小红书登录状态...
爬取数据有账户封禁的风险，建议使用非主账号登录。
登录成功
检查时间: Thu Feb  6 18:40:31 2025
请在文本框中根据提示输入搜索关键词和笔记爬取数量。
即将开始检查网页加载状态...
如果网页进入人机验证页面，请先手动完成验证。
加载成功
检查时间: Thu Feb  6 18:40:48 2025


In [9]:
def change_mode(browser):
    # 更改模式为图文
    try:
        mode_button = browser.find_element(By.XPATH, '//*[@id="search-type"]/div/div/div[2]')
        mode_button.click()
        print('已自动更改模式为图文。')
    except Exception as e:
        print(f"更改模式失败: {e}")

selected_order_text = ''
def change_sort_order(browser, action):
    # 更改排序方式
    sort_order = {
        '综合': 1,
        '最新': 2,
        '最热': 3
    }
    print("请选择排序方式:")
    for idx, order in sort_order.items():
        print(f'{order}. {idx}')
    
    try:
        global selected_order_text
        selected_order_text = input("请输入排序方式对应的名称: ").strip()
        if selected_order_text not in sort_order:
            print("请输入有效的排序方式...")
            return
        
        selected_order_index = sort_order[selected_order_text]
    except Exception as e:
        print(f"处理排序选择时出错: {e}")
        return

    try:
        element = browser.find_element(By.XPATH, '//*[@id="global"]/div[2]/div[2]/div/div[1]/div[2]')
        action.move_to_element(element).perform()# 模拟鼠标悬停
        menu = browser.find_element(By.CLASS_NAME, 'dropdown-items')
        option = menu.find_element(By.XPATH, f'/html/body/div[4]/div/li[{selected_order_index}]')
        option.click()# 模拟鼠标点击

        print('已选择排序方式为:',selected_order_text)
        print('检查时间:',time.ctime())

    except Exception as e:
        print(f"更改排序方式失败: {e}")

change_mode(browser)
change_sort_order(browser, action)

已自动更改模式为图文。
请选择排序方式:
1. 综合
2. 最新
3. 最热
请输入有效的排序方式...


In [10]:
# 初始化数据存储列表
authorName_list = []
likeNr_list = []
URL_list = []
userURL_list = []
commentNr_list = []
post_title_list = [] 
caption_list = []  # 用于存储帖子内容
datePublished_list = []
images_list = []
author_avatar_list = []  # 用于存储头像图片
starNr_list = []
authorCollectNr_list = []
authorFansNr_list = []
authorNoteNr_list = []
video_urls = [] 

def parsePage(page_source):
    """
    解析当前页面的HTML内容，提取笔记的基本信息并更新对应的列表。
    """
    response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
    selector = Selector(response)
    print("正在分析页面结构...")

    containers = ['//div[contains(@class, "note-item")]']
    for container in containers:
        elements = selector.xpath(container)
        if elements:
            print(f"找到容器，xpath: {container}")

    content_elements = selector.xpath('//section[contains(@class, "note-item")]')
    if content_elements:
        print(f"找到 {len(content_elements)} 个内容元素")
        for element in content_elements:
            try:
                # 提取用户URL
                user_url = element.xpath('.//a[contains(@class, "cover")]/@href').get()
                if user_url:
                    note_id = user_url.split('/')[-1].split('?')[0]
                    if note_id in URL_list:
                        continue  # 避免重复
                    URL_list.append(note_id)
                    userURL_list.append(user_url)

                    # 提取作者名字
                    author = element.xpath('.//div[contains(@class, "author-wrapper")]//span[contains(@class, "name")]/text()').get()
                    authorName_list.append(author.strip() if author else "N/A")

                    # 提取点赞数量
                    likes = element.xpath('.//span[contains(@class, "like-wrapper")]/span[contains(@class, "count")]/text()').get()
                    likeNr_list.append(likes.strip() if likes else "0")

                    # 提取帖子标题
                    post_title = element.xpath('.//a[contains(@class, "title")]//span/text()').getall()
                    post_title_cleaned = ' '.join([c.strip() for c in post_title if c.strip()])
                    post_title_list.append(post_title_cleaned if post_title_cleaned else "N/A")

                    # 提取图片（主图）
                    main_image = element.xpath('.//a[contains(@class, "cover")]/img/@src').get()
                    images_list.append(main_image.strip() if main_image else "N/A")

                    # 提取头像图片
                    avatar_image = element.xpath('.//a[contains(@class, "author")]/img/@src').get()
                    author_avatar_list.append(avatar_image.strip() if avatar_image else "N/A")

                    # 初始化附加字段的默认值
                    commentNr_list.append("0")
                    datePublished_list.append("N/A")
                    starNr_list.append("0")
                    authorCollectNr_list.append("0")
                    authorFansNr_list.append("0")
                    authorNoteNr_list.append("0")
                    video_urls.append("N/A")  
                    caption_list.append("N/A")  # 默认值

                    qbar.update(1)
            except Exception as e:
                print(f"处理元素时出错: {str(e)}")
                continue
    print(f"当前已爬取总数: {len(URL_list)}")

def extract_additional_fields(note_url, note_id):
    """
    访问每个笔记的页面，提取附加字段，包括评论数量、发布时间、收藏数量、粉丝数量、笔记数量、视频URL以及帖子内容（Caption）。
    """
    try:
        full_note_url = f'https://www.xiaohongshu.com{note_url}'
        browser.get(full_note_url)
        
        # 显式等待meta标签加载完成
        wait = WebDriverWait(browser, 15)
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@name="description"]')))
        
        page_source = browser.page_source
        response = TextResponse(url=browser.current_url, body=page_source.encode('utf-8'), encoding='utf-8')
        selector = Selector(response)

        # 提取各字段
        comments = selector.xpath('//*[@class="total"]/text()').get()
        comments = comments.strip() if comments else "0"

        date_published = selector.xpath('//*[@class="date"]/text()').get()
        if not date_published:
            date_published = selector.xpath('//time/@datetime').get()
        date_published = date_published.strip() if date_published else "N/A"

        stars = selector.xpath('//*[@class="count"]/text()').get()
        stars = stars.strip() if stars else "0"

        collect_nr = selector.xpath('//span[contains(@class, "collect") or contains(@class, "saved")]/text()').get()
        collect_nr = collect_nr.strip() if collect_nr else "0"

        fans_nr = selector.xpath('//span[contains(@class, "fans") or contains(@class, "followers")]/text()').get()
        fans_nr = fans_nr.strip() if fans_nr else "0"

        note_nr = selector.xpath('//span[contains(@class, "notes") or contains(@class, "posts")]/text()').get()
        note_nr = note_nr.strip() if note_nr else "0"

        video_url = selector.xpath('//video/@src').get()
        video_url = video_url.strip() if video_url else "N/A"

        caption = selector.xpath('//*[@name="description"]/@content').get()
        caption = caption.strip() if caption else "N/A"

        # 更新全局列表
        if note_id in URL_list:
            index = URL_list.index(note_id)
            commentNr_list[index] = comments
            datePublished_list[index] = date_published
            starNr_list[index] = stars
            authorCollectNr_list[index] = collect_nr
            authorFansNr_list[index] = fans_nr
            authorNoteNr_list[index] = note_nr
            video_urls[index] = video_url
            caption_list[index] = caption
            print(f"已提取附加字段，笔记ID: {note_id}")
        else:
            print(f"笔记ID {note_id} 未在URL_list中找到。")
    except Exception as e:
        print(f"提取附加字段时出错，笔记ID: {note_id}, 错误: {str(e)}")

def ensure_search_results():
    """
    确保已导航到搜索结果页面，并选择“图文”模式。
    """
    current_url = browser.current_url
    print(f"当前URL: {current_url}")

    search_url = f'https://www.xiaohongshu.com/search_result?keyword={key_word}&source=web_explore_feed'
    browser.get(search_url)

    try:
        wait = WebDriverWait(browser, 15)
        wait.until(EC.title_contains(key_word))
        print("已导航到搜索结果页面。")
    except:
        print("导航到搜索结果页面时超时。")
        browser.quit()
        exit()

    try:
        selectors = [
            "//div[text()='图文']",
            "//div[contains(@class, 'tab')]//span[text()='图文']",
            "//div[contains(@class, 'filter')]//div[text()='图文']",
            "//*[contains(text(), '图文')]"
        ]
        for sel in selectors:
            try:
                element = WebDriverWait(browser, 10).until(
                    EC.element_to_be_clickable((By.XPATH, sel))
                )
                print(f"找到元素，选择器: {sel}")
                element.click()
                time.sleep(2)
                break
            except:
                continue
        else:
            print("未找到“图文”标签，可能页面结构已更改。")
    except Exception as e:
        print(f"切换视图时出错: {e}")

    time.sleep(3)

def download_image(url, save_path):
    """
    下载图片并保存到指定路径。
    """
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        print(f"下载图片时出错，URL: {url}, 错误: {str(e)}")
        return False

def create_directories():
    """
    创建用于存储主图片和头像图片的目录。
    """
    if not os.path.exists('images_keyword'):
        os.makedirs('images_keyword')
    if not os.path.exists('avatars_keyword'):
        os.makedirs('avatars_keyword')

# 创建图片保存目录
create_directories()

# 定义进度条：用于实时跟踪已爬取的笔记数量
qbar = tqdm(total=num, desc="爬取进度")

ensure_search_results()

while len(URL_list) < num:
    for _ in range(3):
        browser.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)
    parsePage(browser.page_source)
    if '- THE END -' in browser.page_source or 'No more content' in browser.page_source:
        print(f"已到达内容末尾。总共收集: {len(URL_list)} 条")
        break
    time.sleep(random.uniform(2, 4))

print(f"总共收集的条目数: {len(URL_list)}")
if URL_list:
    print("收集的数据样本:")
    for i in range(min(5, len(URL_list))):
        print(f"URL: {URL_list[i]}")

if len(URL_list) > num:
    URL_list = URL_list[:num]
    authorName_list = authorName_list[:num]
    likeNr_list = likeNr_list[:num]
    userURL_list = userURL_list[:num]
    commentNr_list = commentNr_list[:num]
    post_title_list = post_title_list[:num]
    datePublished_list = datePublished_list[:num]
    images_list = images_list[:num]
    author_avatar_list = author_avatar_list[:num]
    starNr_list = starNr_list[:num]
    authorCollectNr_list = authorCollectNr_list[:num]
    authorFansNr_list = authorFansNr_list[:num]
    authorNoteNr_list = authorNoteNr_list[:num]
    video_urls = video_urls[:num]
    caption_list = caption_list[:num]

print(f"截断后的总条目数: {len(URL_list)}")
print("收集的数据样本:")
for i in range(min(5, len(URL_list))):
    print(f"作者: {authorName_list[i]}, 点赞: {likeNr_list[i]}, URL: {URL_list[i]}")
qbar.close()

# 提取附加字段（帖子内容、日期发布、评论数量等）
print("开始提取附加字段，包括帖子内容、日期发布和评论数量...")
qbar = tqdm(total=len(URL_list), desc="已获取的笔记数量...")
for note_id, note_url in zip(URL_list, userURL_list):
    extract_additional_fields(note_url, note_id)
    qbar.update(1)
    time.sleep(random.uniform(2, 4))
qbar.close()

# 下载图片
print("开始下载主图片和头像图片...")

# 下载主图片
image_download_bar = tqdm(total=len(images_list), desc="下载主图片")
for idx, image_url in enumerate(images_list):
    if image_url == "N/A":
        image_download_bar.update(1)
        continue
    image_extension = os.path.splitext(image_url)[1].split('?')[0]
    if image_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        image_extension = '.jpg'
    image_filename = f"images_keyword/{URL_list[idx]}{image_extension}"
    success = download_image(image_url, image_filename)
    if not success:
        image_filename = "N/A"
    images_list[idx] = image_filename
    image_download_bar.update(1)
image_download_bar.close()

# 下载头像图片
avatar_download_bar = tqdm(total=len(author_avatar_list), desc="下载头像图片")
for idx, avatar_url in enumerate(author_avatar_list):
    if avatar_url == "N/A":
        avatar_download_bar.update(1)
        continue
    avatar_extension = os.path.splitext(avatar_url)[1].split('?')[0]
    if avatar_extension.lower() not in ['.jpg', '.jpeg', '.png', '.gif']:
        avatar_extension = '.jpg'
    avatar_filename = f"avatars_keyword/{URL_list[idx]}{avatar_extension}"
    success = download_image(avatar_url, avatar_filename)
    if not success:
        avatar_filename = "N/A"
    author_avatar_list[idx] = avatar_filename
    avatar_download_bar.update(1)
avatar_download_bar.close()

print("所有图片下载完成。")

# 创建数据字典
data = {
    'Author Name': authorName_list,
    'Likes': likeNr_list,
    'Comments': commentNr_list,
    'Post Title': post_title_list, 
    'Caption': caption_list,
    'Date Published': datePublished_list,
    'Images': images_list,
    'Author Avatar': author_avatar_list,
    'Stars': starNr_list,
    'Author Collect Nr': authorCollectNr_list,
    'Author Fans Nr': authorFansNr_list,
    'Author Note Nr': authorNoteNr_list,
    'Video URL': video_urls,
    'URL': URL_list,
    'User URL': userURL_list
}

df = pd.DataFrame(data)
df.set_index('URL', inplace=True)
print(df.head())
df.to_csv('scraped_xhs_keyword.csv', encoding='utf-8-sig')
print("数据已保存到 'scraped_xhs_keyword.csv'")

爬取进度:   0%|          | 0/5 [00:00<?, ?it/s]

当前URL: https://www.xiaohongshu.com/search_result?keyword=%E9%BB%84%E5%AD%90%E9%9F%AC&source=web_explore_feed&type=51
已导航到搜索结果页面。
找到元素，选择器: //*[contains(text(), '图文')]
正在分析页面结构...
找到容器，xpath: //div[contains(@class, "note-item")]
找到 18 个内容元素
当前已爬取总数: 16
总共收集的条目数: 16
收集的数据样本:
URL: 67a47523000000001902d710
URL: 67a4d3500000000029013e39
URL: 67a4daa60000000029030827
URL: 67a540a80000000029039ff2
URL: 67a4c6f8000000001800680d
截断后的总条目数: 5
收集的数据样本:
作者: 韬韬日常分享, 点赞: 92, URL: 67a47523000000001902d710
作者: 聆听（拥有洋韬亲签版）, 点赞: 868, URL: 67a4d3500000000029013e39
作者: tuantuan, 点赞: 803, URL: 67a4daa60000000029030827
作者: rintuesday, 点赞: 赞, URL: 67a540a80000000029039ff2
作者: 玉, 点赞: 26, URL: 67a4c6f8000000001800680d
开始提取附加字段，包括帖子内容、日期发布和评论数量...


已获取的笔记数量...:   0%|          | 0/5 [00:00<?, ?it/s]

已提取附加字段，笔记ID: 67a47523000000001902d710
已提取附加字段，笔记ID: 67a4d3500000000029013e39
已提取附加字段，笔记ID: 67a4daa60000000029030827
已提取附加字段，笔记ID: 67a540a80000000029039ff2
已提取附加字段，笔记ID: 67a4c6f8000000001800680d
开始下载主图片和头像图片...


下载主图片:   0%|          | 0/5 [00:00<?, ?it/s]

下载头像图片:   0%|          | 0/5 [00:00<?, ?it/s]

所有图片下载完成。
                          Author Name Likes  Comments            Post Title  \
URL                                                                           
67a47523000000001902d710       韬韬日常分享    92   共 7 条评论           黄子韬小两口的抽风日常   
67a4d3500000000029013e39  聆听（拥有洋韬亲签版）   868  共 58 条评论        网友问黄了桃带几个女孩来马代   
67a4daa60000000029030827     tuantuan   803  共 78 条评论  黄子韬徐艺洋半夜一起发疯太搞笑了哈哈哈哈   
67a540a80000000029039ff2   rintuesday     赞         0        黄子韬《皇冠》十周年啦～～～   
67a4c6f8000000001800680d            玉    26   共 1 条评论          怎么办啊，黄子韬真的好好   

                                                                    Caption  \
URL                                                                           
67a47523000000001902d710            #搞笑 #黄子韬 #搞笑段子 #黄子韬 #黄子韬直播 #徐艺洋 #徐艺洋黄子韬   
67a4d3500000000029013e39        太搞笑了，晚上一看他们直播就激动的睡不着 #黄子韬  #徐艺洋 #直播切片 #马尔代夫   
67a4daa60000000029030827             #黄子韬 #黄子韬🍑 #徐艺洋 #黄子韬徐艺洋 #黄子韬直播 #徐艺洋黄子韬   
67a540a80000000029039ff2                 